In [1]:
filename = 'iris.data'
f = open(filename,'r')

datapoints = []

for x in f:
    temp_ls = []
    temp = ''
    for ch in x:
        if (ch >= '0' and ch <='9') or (ch == '.'):
#             print('ch:' + ch +' temp:' + temp)
            temp += ch
        elif ch == ',':
#             print('temp:' + temp)
            temp_ls.append(float(temp))
            temp = ''
        else:
            if(temp_ls != []):
                datapoints.append(temp_ls)
            break

# print(datapoints)

In [2]:
import math
def distance(point_a,point_b):
    sum = 0
    for i in range(len(point_a)):
        sum += (point_a[i]-point_b[i])**2
    
    return math.sqrt(sum)

In [3]:
def get_core_points(epsilon,min_pts):
    core_points = []

    for point_a in datapoints:
        neighbors = []
        for point_b in datapoints:
            if distance(point_a,point_b) < epsilon:
                neighbors.append(point_b)

        if len(neighbors) > min_pts:
            core_points.append(point_a)
    
    return core_points

In [4]:
def density_connected(core_point,cluster_index,cluster_id,core_points):
    neighbors = []
    for point_a in datapoints:
        if (distance(point_a,core_point) < epsilon) and (point_a != core_point):
            neighbors.append(point_a)
    
    for y in neighbors:
        if cluster_id[tuple(y)] == -1:
#             print('neighbor:' + str(y))
            cluster_id[tuple(y)] = cluster_index
            if y in core_points:
#                 print('is core point')
                density_connected(y,cluster_index,cluster_id,core_points)
            
    

In [5]:
def assign_clusters(core_points):
    cluster_id = {}

    for point in datapoints:
        cluster_id[tuple(point)] = -1

    cluster_index = 0

    for point in core_points:
#         print('point:' + str(point))
        if cluster_id[tuple(point)] == -1:
            cluster_index += 1
            cluster_id[tuple(point)] = cluster_index
            density_connected(point,cluster_index,cluster_id,core_points)
    
    return cluster_id

In [6]:
def create_clusters(cluster_id):
    clusters = {}
    for point in datapoints:
        if cluster_id[tuple(point)] not in clusters.keys():
            clusters[cluster_id[tuple(point)]] = []
            clusters[cluster_id[tuple(point)]].append(point)
        else:
            clusters[cluster_id[tuple(point)]].append(point)
    
    return clusters

In [7]:
def perform_dbscan(epsilon,min_pts):
    core_points = get_core_points(epsilon,min_pts)
    cluster_id = assign_clusters(core_points)
    clusters = create_clusters(cluster_id)
    return clusters

In [8]:
def calculate_sse(useful_clusters):
    sse = 0
    for index in useful_clusters:
        temp_cluster = useful_clusters[index]
        dimension = len(temp_cluster[0])
        temp_sum = []
        for i in range(dimension):
            temp_sum.append(0)
        
        for point in temp_cluster:
            for j in range(dimension):
                temp_sum[j] += point[j]
        
        temp_mean = []
        
        for j in range(dimension):
            temp_mean.append(temp_sum[j]/len(temp_cluster))
        
        
        diff = 0
        
        for point in temp_cluster:
            diff = 0
            for j in range(dimension):
                diff += temp_mean[j] - point[j]
            
            sse += diff**2
        
    return sse
            

In [9]:
import time

epsilon = 0.5
min_pts = 3

start = time.time()

final_clusters = perform_dbscan(epsilon,min_pts)

end = time.time()
useful_clusters = {}
noise = {}
for c in final_clusters:
    if c == -1:
        noise = final_clusters[c]
    else:
        useful_clusters[c] = final_clusters[c]

sse = calculate_sse(useful_clusters)
print(sse)
print('Number of clusters:' + str(len(useful_clusters)))
print('Number of noise points:' + str(len(noise)))
print('time taken:' + str(end-start))

232.40061224489784
Number of clusters:3
Number of noise points:13
time taken:0.08380675315856934
